In [ ]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

af = getDF('/content/AMAZON_FASHION_5.json.gz')

In [ ]:
af

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
1,5.0,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Black (3746...",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
2,5.0,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue/Gray L...",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
3,5.0,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue (37867...",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
4,5.0,True,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,"{'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}",Tonya B.,Great product and price!,Five Stars,1441324800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3171,5.0,True,"07 2, 2018",A2077NII5H62R2,B005AGO4LU,"{'Size:': ' 8.5 B(M) US', 'Color:': ' Green Gl...",Amazon Customer,Perfect fit!,Five Stars,1530489600,NaN,NaN
3172,5.0,True,"06 28, 2018",A2IBS6PIPAGAB5,B005AGO4LU,"{'Size:': ' 5 B(M) US', 'Color:': ' Wolf Grey/...",J. Avila,My favorite cross trainers!,Comfortable,1530144000,NaN,NaN
3173,5.0,True,"06 25, 2018",A1GTC5EVSJNCQ8,B005AGO4LU,"{'Size:': ' 8 B(M) US', 'Color:': ' Blue Tint/...",Amazon Customer,Love them fit perfect,Five Stars,1529884800,NaN,NaN
3174,5.0,True,"06 20, 2018",A311XHHLM12MUT,B005AGO4LU,"{'Size:': ' 9 B(M) US', 'Color:': ' Blue Tint/...",Peter,Favorite Nike shoe ever! The flex sole is exce...,Love them!,1529452800,NaN,NaN


In [ ]:
ratings=pd.read_csv('/content/AMAZON_FASHION.csv')

In [ ]:
ratings

,asin,reviewerID,rating,unixReviewTime
0,7106116521,A1D4G1SNUZWQOT,5.0,1.413763e+09
1,7106116521,A3DDWDH9PX2YX2,2.0,1.411862e+09
2,7106116521,A2MWC41EW7XL15,4.0,1.408925e+09
3,7106116521,A2UH2QQ275NV45,2.0,1.408838e+09
4,7106116521,A89F3LQADZBS5,3.0,1.406419e+09
...,...,...,...,...
211087,B00MZIHOCM,A2Y3W4W2JUYYCI,5.0,1.489709e+09
211088,B00MZIHOCM,A2ZZI9TE4SVLYY,4.0,1.489018e+09
211089,B00MZIHOCM,AB32XCBQNH7CJ,5.0,1.488931e+09
211090,B00MZIHOCM,A2A4PBQZMRPV4B,5.0,1.487981e+09


In [46]:
!pip install surprise
import surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163000 sha256=3ad1014828796fc201e17992d3d4d91926931aaf948db294362891d3dd44a217
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [73]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the dataset
data = af

In [65]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np



# Replace NaN values with empty strings in reviewText
data['reviewText'] = data['reviewText'].fillna('')

# Content-based filtering
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['reviewText'])
content_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

# Collaborative filtering
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(data[['reviewerID', 'asin', 'overall']], reader)
trainset, testset = train_test_split(dataset, test_size=0.2)
svd = SVD()
svd.fit(trainset)

# Compute similar items using SVD
item_factors = svd.qi
collab_similarity = item_factors.dot(item_factors.T)

# Resize content_similarity to match the shape of collab_similarity
content_similarity_resized = np.resize(content_similarity, collab_similarity.shape)

# Merge the models
hybrid_similarity = 0.5 * content_similarity_resized + 0.5 * collab_similarity

# Example: recommend products for a given reviewer
reviewer_id = 'ALJ66O1Y6SLHA'
if reviewer_id in data['reviewerID'].values:
    reviewer_idx = data[data['reviewerID'] == reviewer_id].index[0]
    similarities = hybrid_similarity[reviewer_idx]
    similar_indices = similarities.argsort()[::-1][1:]  # Exclude the reviewer's own index
    recommended_products = data.iloc[similar_indices]['asin'].values[:25]
    print(set(recommended_products))
else:
    print(f"Reviewer with ID {reviewer_id} not found in the dataset.")






{'B000KPIHQ4', 'B000K2PJ4K', 'B000V0IBDM', 'B000YFSR5G', 'B000YFSR4W'}


In [67]:
# Given set of ASIN values
asin_values = {'B000KPIHQ4', 'B000K2PJ4K', 'B000V0IBDM', 'B000YFSR5G', 'B000YFSR4W'}

# Filter the DataFrame for the given ASIN values
filtered_products = data[data['asin'].isin(asin_values)]

# Print the filtered products
print(filtered_products[['asin', 'reviewText']])


           asin                                         reviewText
0    B000K2PJ4K                           Great product and price!
1    B000K2PJ4K                           Great product and price!
2    B000K2PJ4K                           Great product and price!
3    B000K2PJ4K                           Great product and price!
4    B000K2PJ4K                           Great product and price!
..          ...                                                ...
129  B000YFSR4W  Nice pants but too small for a medium. Return it.
130  B000YFSR4W  Nice pants but too small for a medium. Returne...
131  B000YFSR4W  Very comfy. If they had pockets, theyd be awes...
132  B000YFSR4W  Was terribly disappointed, the pants were way ...
133  B000YFSR4W                                      As described.

[116 rows x 2 columns]


In [71]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
import numpy as np

# Assuming 'data' is your DataFrame containing the dataset

# Load the dataset
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(data[['reviewerID', 'asin', 'overall']], reader)

# Split the dataset into training and test sets
trainset, testset = train_test_split(dataset, test_size=0.2)

# Train the SVD model
model = SVD()
model.fit(trainset)

# Get predictions for the test set
predictions = model.test(testset)


In [72]:
# Compute precision and recall
precision_sum = 0
recall_sum = 0
n_users = len(set([pred.uid for pred in predictions]))

for uid in set([pred.uid for pred in predictions]):
    user_ratings = [pred for pred in predictions if pred.uid == uid]
    top_n = sorted(user_ratings, key=lambda x: x.est, reverse=True)[:5]  # Top 5 recommendations
    relevant = sum((pred.r_ui >= 4) for pred in top_n)  # Number of relevant items in top 5

    if len(user_ratings) == 0 or relevant == 0:
        continue  # Skip this user if no ratings or no relevant items

    precision_sum += relevant / 5
    recall_sum += relevant / sum((pred.r_ui >= 4) for pred in user_ratings)

precision = precision_sum / n_users
recall = recall_sum / n_users

print(f"Precision: {precision}")
print(f"Recall: {recall}")


Precision: 0.3254545454545461
Recall: 0.8363636363636363
